In [43]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers datasets accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from datasets import load_dataset

# Load the text dataset
dataset = load_dataset("text", data_files="/content/drive/MyDrive/Model/formatted_data.txt")


Generating train split: 0 examples [00:00, ? examples/s]

In [2]:
cd


/root


In [5]:
from transformers import GPT2Tokenizer

# Load GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Set EOS token as pad token
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

# Apply the tokenization function to the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Check the special tokens map (optional)
print(tokenizer.special_tokens_map)


Map:   0%|          | 0/6318 [00:00<?, ? examples/s]

{'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}


In [8]:
tokenized_dataset.save_to_disk('/content/drive/MyDrive/Model')


Saving the dataset (0/1 shards):   0%|          | 0/6318 [00:00<?, ? examples/s]

In [11]:
def tokenize_function(examples):
    # Tokenizing the dataset and shifting the labels
    # The input_ids will be the same as the labels, but shifted by one token
    # So, we create a 'labels' field from the 'input_ids' (shifted by 1 token)
    encodings = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)
    encodings["labels"] = encodings["input_ids"].copy()  # Copy the input_ids to labels
    return encodings


In [12]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/6318 [00:00<?, ? examples/s]

In [14]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments

model = GPT2LMHeadModel.from_pretrained("gpt2")

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Model/checkpoints/gpt2-finetuned",   # Directory to save model checkpoints
    overwrite_output_dir=True,
    num_train_epochs=3,             # Number of epochs
    per_device_train_batch_size=4,  # Reduce if memory issues occur
    save_steps=500,                 # Save model every 500 steps
    save_total_limit=2,             # Keep only the last 2 checkpoints
    logging_dir="/content/drive/MyDrive/Model/checkpoints/logs",           # Directory for logs
    logging_steps=100,
    evaluation_strategy="no",       # Skip evaluation if resources are limited
    warmup_steps=100,
    learning_rate=5e-5,
    weight_decay=0.01,
    fp16=True,                      # Enable mixed precision for faster training
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"]
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
trainer.train()


Step,Training Loss
100,2.022000
200,0.246100
300,0.232900
400,0.194100
500,0.221400
600,0.166000
700,0.173200
800,0.194500
900,0.221500
1000,0.177500


TrainOutput(global_step=4740, training_loss=0.20754243583115847, metrics={'train_runtime': 1410.5563, 'train_samples_per_second': 13.437, 'train_steps_per_second': 3.36, 'total_flos': 4952529174528000.0, 'train_loss': 0.20754243583115847, 'epoch': 3.0})

In [16]:
model.save_pretrained("/content/drive/MyDrive/Model/modelpretrsinded/gpt2-finetuned")
tokenizer.save_pretrained("/content/drive/MyDrive/Model/modelpretrsinded/gpt2-finetuned")


('/content/drive/MyDrive/Model/modelpretrsinded/gpt2-finetuned/tokenizer_config.json',
 '/content/drive/MyDrive/Model/modelpretrsinded/gpt2-finetuned/special_tokens_map.json',
 '/content/drive/MyDrive/Model/modelpretrsinded/gpt2-finetuned/vocab.json',
 '/content/drive/MyDrive/Model/modelpretrsinded/gpt2-finetuned/merges.txt',
 '/content/drive/MyDrive/Model/modelpretrsinded/gpt2-finetuned/added_tokens.json')

In [23]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the model and tokenizer from Google Drive
model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/Model/modelpretrsinded/gpt2-finetuned")
tokenizer = GPT2Tokenizer.from_pretrained("/content/drive/MyDrive/Model/modelpretrsinded/gpt2-finetuned")

# Set pad_token_id to eos_token_id if not already set
model.config.pad_token_id = model.config.eos_token_id

# Test the model by generating text
input_text = "What are the specializations of Dr. Naresh Trehan?,  "
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Generate output
output = model.generate(
    input_ids,
    max_length=100,
    num_return_sequences=1,
    pad_token_id=model.config.pad_token_id  # Explicitly set pad_token_id
)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)


What are the specializations of Dr. Naresh Trehan?,   Dr. Naresh Trehan?  Dr. Naresh Trehan is a Senior Consultant in the Department of Cardiology at Medanta The Medicity. He completed his MBBS from the prestigious Jawaharlal Nehru Medical College, New Delhi and MD from the prestigious Jawaharlal Nehru Medical College, New Delhi. He has over 10 years of experience in the field of Cardiology and has worked as


In [24]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the model and tokenizer from Google Drive
model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/Model/modelpretrsinded/gpt2-finetuned")
tokenizer = GPT2Tokenizer.from_pretrained("/content/drive/MyDrive/Model/modelpretrsinded/gpt2-finetuned")

# Set pad_token_id to eos_token_id if not already set
model.config.pad_token_id = model.config.eos_token_id

# Test the model by generating text
input_text = "Dr. Naresh Trehan is a world-renowned cardiovascular surgeon."
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Generate output
output = model.generate(
    input_ids,
    max_length=100,
    num_return_sequences=1,
    pad_token_id=model.config.pad_token_id  # Explicitly set pad_token_id
)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)


Dr. Naresh Trehan is a world-renowned cardiovascular surgeon. He has done his MBBS from the prestigious Jawaharlal Nehru Medical College, New Delhi and MD from the prestigious Jawaharlal Nehru Medical College, New Delhi. He has also done his M.Ch in Cardiology from the prestigious Jawaharlal Nehru Medical College, New Delhi.


In [28]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load the model and tokenizer from your fine-tuned model
model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/Model/modelpretrsinded/gpt2-finetuned")
tokenizer = GPT2Tokenizer.from_pretrained("/content/drive/MyDrive/Model/modelpretrsinded/gpt2-finetuned")

# Test prompt
input_text = "What are the specializations of Dr. Naresh Trehan?"

# Encode input text and get the prediction
input_ids = tokenizer.encode(input_text, return_tensors='pt')
target_ids = input_ids.clone()

# Ensure the model is in evaluation mode
model.eval()

# Get model output
with torch.no_grad():
    outputs = model(input_ids, labels=target_ids)
    loss = outputs.loss  # Calculate the loss

# Calculate perplexity
perplexity = torch.exp(loss)
print(f"Perplexity: {perplexity.item()}")


Perplexity: 44.35853958129883
